In [14]:
# !pip install python-binance
# !pip install python-dotenv
# !pip install pandas_ta

In [28]:
from binance.client import Client
import pandas as pd
from dotenv import load_dotenv 
import os
import pandas_ta as ta
from sklearn.preprocessing import StandardScaler

In [38]:
from sklearn.model_selection import train_test_split # Para dividir os dados
from sklearn.ensemble import RandomForestClassifier # O algoritmo de Floresta Aleatória
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score # Métricas de avaliação


In [3]:
load_dotenv()

API_KEY = os.getenv("API_KEY")
API_SECRET = os.getenv("API_SECRET")

In [32]:
if not API_KEY or not API_SECRET:
    print("Erro: As chaves API_KEY ou API_SECRET não foram carregadas do arquivo .env.")
    print("Verifique se o arquivo .env existe na mesma pasta e se as variáveis estão corretas.")
    exit() # Sai do programa se as chaves não forem encontradas

# 4. Inicializa o cliente da Binance (mesmo código de antes)
try:
    client = Client(API_KEY, API_SECRET)
    print("Conexão com a Binance estabelecida com sucesso!")
except Exception as e:
    print(f"Erro ao conectar com a Binance: {e}")
    print("Verifique suas chaves de API e sua conexão com a internet.")
    exit()

# 5. Define os parâmetros para a busca dos dados de candlestick (mesmo código de antes)
symbol = 'BTCUSDT'
interval = '1h'
limit = 2500

print(f"\nBuscando os últimos {limit} candlesticks de {symbol} no intervalo de {interval}...")


Conexão com a Binance estabelecida com sucesso!

Buscando os últimos 2500 candlesticks de BTCUSDT no intervalo de 1h...


In [5]:
# 6. Faz a requisição para a API da Binance
klines = client.get_historical_klines(symbol, interval, limit=limit)


In [33]:
# 7. Processa os dados brutos e os organiza em um DataFrame do Pandas
data = []
for kline in klines:
    data.append({
        'timestamp': kline[0],
        'open': float(kline[1]),
        'high': float(kline[2]),
        'low': float(kline[3]),
        'close': float(kline[4]),
        'volume': float(kline[5])
    })

df = pd.DataFrame(data)
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
df.set_index('timestamp', inplace=True)

# Armazenar o DataFrame original antes de renomear colunas
# para que possamos usar 'close' para o cálculo do label
df_original_cols = df.copy()

# Assegura que as colunas numéricas estão no tipo correto para pandas_ta
df.columns = ['Open', 'High', 'Low', 'Close', 'Volume']

In [18]:
# --- 8. Adicionar Indicadores Técnicos (Feature Engineering) ---
print("\nAdicionando indicadores técnicos ao DataFrame...")

# Média Móvel Simples (SMA) de 20 períodos
# df.ta.sma(length=20, append=True) adiciona a coluna 'SMA_20' diretamente ao DataFrame
df.ta.sma(length=20, append=True)

# Bandas de Bollinger (BBands)
# bb_df = df.ta.bbands(append=True) adiciona várias colunas: 'BBL_5_2.0', 'BBM_5_2.0', 'BBU_5_2.0', etc.
df.ta.bbands(append=True)

# Índice de Força Relativa (RSI) de 14 períodos
df.ta.rsi(length=14, append=True)

print("Indicadores adicionados com sucesso!")


Adicionando indicadores técnicos ao DataFrame...
Indicadores adicionados com sucesso!


In [25]:
# --- 9 Tratamento de Valores NaN ---
print("\nTratando valores NaN...")

# Guarda o número de linhas antes de remover os NaNs
initial_rows = len(df)

# Remove todas as linhas que contêm qualquer valor NaN
df.dropna(inplace=True)

# Guarda o número de linhas depois de remover os NaNs
final_rows = len(df)

print(f"Número de linhas antes do tratamento de NaN: {initial_rows}")
print(f"Número de linhas após o tratamento de NaN: {final_rows}")
print(f"Total de linhas removidas: {initial_rows - final_rows}")


Tratando valores NaN...
Número de linhas antes do tratamento de NaN: 500
Número de linhas após o tratamento de NaN: 481
Total de linhas removidas: 19


In [29]:
### **10. Normalização/Escalonamento de Dados (StandardScaler)**
print("\nRealizando o escalonamento dos dados (StandardScaler)...")
# Certifique-se de que todas as colunas sejam numéricas antes de escalonar
features_to_scale = df.columns.tolist()
# Cria uma instância do StandardScaler
scaler = StandardScaler()
# O resultado é um array NumPy, então o convertemos de volta para um DataFrame.
df_scaled = pd.DataFrame(scaler.fit_transform(df[features_to_scale]),
                         columns=features_to_scale,
                         index=df.index)

print("Escalonamento concluído!")


Realizando o escalonamento dos dados (StandardScaler)...
Escalonamento concluído!


In [39]:
# --- 10. Criação dos Labels (Comprar/Vender/Manter) ---
print("\nCriando os labels de Comprar/Vender/Manter...")

look_forward_period = 5
price_change_threshold = 0.005

df['future_close'] = df['Close'].shift(-look_forward_period)
df['price_change'] = (df['future_close'] - df['Close']) / df['Close']

def get_label(change, threshold):
    if change > threshold:
        return 1  # Comprar
    elif change < -threshold:
        return -1 # Vender
    else:
        return 0  # Manter

df['label'] = df['price_change'].apply(lambda x: get_label(x, price_change_threshold))

df.drop(columns=['future_close', 'price_change'], inplace=True)
df.dropna(inplace=True) # Remove as últimas linhas que não têm um 'future_close'

print("Labels criados com sucesso!")


Criando os labels de Comprar/Vender/Manter...
Labels criados com sucesso!


In [40]:
# 11. Dividir em Features (X) e Labels (y) e aplicar escalonamento
X = df.drop(columns=['label'])
y = df['label']

# Agora, escalonar as features (X)
features_to_scale_final = X.columns.tolist()
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X[features_to_scale_final]),
                        columns=features_to_scale_final,
                        index=X.index)

# Verificação final de NaNs após todo o pré-processamento
if X_scaled.isnull().sum().sum() > 0 or y.isnull().sum() > 0:
    print("AVISO: Ainda existem NaNs em X_scaled ou y após o pré-processamento. Verifique!")
    print(X_scaled.isnull().sum())
    print(y.isnull().sum())
    exit() # Interrompe o processo se houver NaNs restantes

In [36]:
# 13. Exibir os primeiros e últimos dados das Features Escalonadas e Labels
print("\nPrimeiras 5 linhas das Features ESCALONADAS (X):")
print(X_scaled.head())

print("\nÚltimas 5 linhas das Features ESCALONADAS (X):")
print(X_scaled.tail())

print("\nPrimeiras 5 linhas dos Labels (y):")
print(y.head())

print("\nÚltimas 5 linhas dos Labels (y):")
print(y.tail())

print(f"\nDataFrame de Features (X) criado com {len(X_scaled)} linhas e {len(X_scaled.columns)} colunas.")
print(f"Series de Labels (y) criada com {len(y)} elementos.")



Primeiras 5 linhas das Features ESCALONADAS (X):
                         Open      High       Low     Close    Volume
timestamp                                                            
2025-07-04 21:00:00 -1.808420 -1.794964 -1.776291 -1.767656 -0.658310
2025-07-04 22:00:00 -1.758144 -1.718464 -1.730263 -1.688546 -0.482024
2025-07-04 23:00:00 -1.679291 -1.723990 -1.713631 -1.750847 -0.757669
2025-07-05 00:00:00 -1.741385 -1.730884 -1.714316 -1.702111 -0.766956
2025-07-05 01:00:00 -1.692813 -1.737359 -1.728447 -1.751562 -0.817997

Últimas 5 linhas das Features ESCALONADAS (X):
                         Open      High       Low     Close    Volume
timestamp                                                            
2025-07-25 12:00:00  0.221976  0.181368  0.157573  0.118263  0.876262
2025-07-25 13:00:00  0.121591  0.168445  0.064265  0.143649  1.640418
2025-07-25 14:00:00  0.146894  0.097435 -0.089234 -0.145463  1.972078
2025-07-25 15:00:00 -0.141269 -0.002454 -0.087252  0.031068  0

In [37]:
# 14. Salvar os DataFrames processados (X_scaled e y)
# É comum salvar as features e labels separadamente
X_scaled_filename = f'{symbol}_{interval}_features_escalonadas.csv'
y_filename = f'{symbol}_{interval}_labels.csv'

X_scaled.to_csv(X_scaled_filename, index=True)
y.to_csv(y_filename, index=True, header=True) # header=True para salvar o nome da coluna 'label'

print(f"\nFeatures escalonadas salvas com sucesso em '{X_scaled_filename}'")
print(f"Labels salvos com sucesso em '{y_filename}'")


Features escalonadas salvas com sucesso em 'BTCUSDT_1h_features_escalonadas.csv'
Labels salvos com sucesso em 'BTCUSDT_1h_labels.csv'
